In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from dataclasses import dataclass

# Sorta Final

In [43]:
class Material:
    def __init__(self, E: float, nu: float, rho: float):
        self.E = E
        self.rho = rho
        self.nu = nu
        self.G = E / (2 * (1 + nu))

class CuboidalBeam:
    def __init__(self, length: float, breadth: float, width: float):
        self.length = length
        self.breadth = breadth
        self.width = width
        self.facearea = breadth*width

    def constraint(self, constype: str = 'fixed', location: float = 'left'):
        """
        constype: 'fixed'
        location: 'left', 'right'
        """
        self.constype = constype
        self.consloc = location

class CylindricalBeam:
    def __init__(self, length: float, radius: float, width: float):
        self.length = length
        self.radius = radius
        # self.width = width
        self.facearea = np.pi*radius**2
        self.polar_moi = (np.pi/2)*radius**4  # polar moment of inertia

    def constraint(self, constype: str = 'fixed', location: float = 'left'):
        """
        constype: 'fixed'
        location: 'left', 'right'
        """
        self.constype = constype
        self.consloc = location
        

class AxialLoad:
    def __init__(self, value: float, location: float):
        '''Assume x along length, and give location in terms of x'''
        self.force = value
        self.location = location
    pass

class TorsionalLoad:
    def __init__(self, value: float, location: float):
        '''Assume x along length, and give location in terms of x'''
        self.torque = value
        self.location = location
        
        
        
#===================================================
@dataclass
class StateOfStress:
    def __init__(self, nx: int = 100, ny: int = 100, nz: int = 100):
        # Initialize a 3x3 stress tensor
        self.stress_tensor = np.zeros((nx, ny, nz, 3, 3))
    
    def update(self, coords: np.ndarray, stress_values: np.ndarray):
        self.stress_tensor[tuple(coords)] = stress_values


class Body:
    def __init__(self, material: Material, geometry: CuboidalBeam | CylindricalBeam, nx: int = 100, ny: int = 100, nz: int = 100, *loads: AxialLoad | TorsionalLoad):
        self.material = material
        self.geometry = geometry
        self.load = load
        
        self.nx = nx
        self.ny = ny
        self.nz = nz
        
        self.results = {}
        self.state_of_stress = StateOfStress(nx,ny,nz)
        
        for load in loads:
            if isinstance(load, AxialLoad):
                self.axl()
            if isinstance(load, TorsionalLoad):
                self.tors()
            
    def axl(self,):
        # calc sigma_xx
        x = np.linspace(0, self.geometry.length, self.nx)
        if self.geometry.consloc == 'left':
            reaction = np.piecewise(
                x,
                [(x >= 0) & (x <= float(self.load.location))],
                [self.load.force])

        if self.geometry.consloc == 'right':
            reaction = np.piecewise(
                x,
                [(x >= float(self.load.location)) & (
                    x <= float(self.geometry.length))],
                [self.load.force])

        axial_stress = reaction/(self.geometry.facearea)

        self.results['x'] = x
        self.results['axial_stress'] = axial_stress
        self.results['internal_force'] = reaction
        self.state_of_stress.update([x,0,0], axial_stress)

    def tors(self,):
        # calculate tau_xz/tau_xy
        # calc sigma_xx
        x = np.linspace(0, self.geometry.radius, 100)
        y,z = np.meshgrid(
            np.linspace(-self.geometry.radius/2, self.geometry.radius/2, self.ny),
            np.linspace(-self.geometry.radius/2, self.geometry.radius/2, self.nz),
            indexing='ij')
        
        if self.geometry.cons[1] == 'left':
            reaction_torque = np.piecewise(
                [x],
                [(x >= 0) & (x <= float(self.load.location))],
                [self.load.torque])

        if self.geometry.cons[1] == 'right':
            reaction_torque = np.piecewise(
                x,
                [(x >= float(self.load.location)) & (
                    x <= float(self.geometry.length))],
                [self.load.torque])

        # torsional_stress = Ticap cross yjcap+zkcap / J 
        # = T(ykcap - zjcap) / J
        tau_xy = -reaction_torque*z/self.geometry.polar_moi
        tau_xz = reaction_torque*y/self.geometry.polar_moi

        sos = np.array([[0, tau_xy, tau_xz], [tau_xy, 0, 0], [tau_xz, 0, 0]])
        # self.results['x'] = x
        # self.results['axial_stress'] = axial_stress
        # self.results['internal_force'] = reaction
        self.state_of_stress.update([x, y, z], sos)


    def plot_result(self,):
        if not self.results:
            raise ValueError("No results to plot. Run analysis first.")

        fig, axes = plt.subplots(2, 1, figsize=(10, 10))

        # Stress plot
        axes[0].plot(self.results['x'],
                     self.results['axial_stress'] / 1e6, 'b-', linewidth=2)
        axes[0].set_xlabel('Position along beam (m)')
        axes[0].set_ylabel('Axial Stress (MPa)')
        axes[0].set_title('Axial Stress Distribution')
        axes[0].grid(True, alpha=0.3)
        # Internal force plot
        axes[1].plot(self.results['x'],
                     self.results['internal_force'], 'r-', linewidth=2)
        axes[1].set_xlabel('Position along beam (m)')
        axes[1].set_ylabel('Internal Force (N)')
        axes[1].set_title('Internal Force Distribution')
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def print_results(self,):
        if not self.results:
            raise ValueError("No results to print. Run analysis first.")

In [11]:
# Define material (Steel)
steel = Material(E=200e9, nu=0.3, rho=7850)

# Define geometry
beam = CuboidalBeam(length=1.0, breadth=0.05, width=0.02)
beam.constraint(constype='fixed', location='left')

# Define load
load = AxialLoad(value=10000, location=0.5)  # 10 kN tension

# Create body and analyze
body = Body(material=steel, geometry=beam, load=load)

# Print summary
# body.print_summary()

# Plot results
body.plot_result()

ValueError: setting an array element with a sequence.

# Playground

In [39]:
stress_field = np.zeros((2, 2, 2, 3, 3))
coords = [0,1,0]
# stress_field[0, 1, 0] = np.eye(3)  # Three separate indices
stress_field[tuple(coords)] = np.eye(3)
stress_field[0,0,0]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
class body:
    class cylinder:
        def __init__(self, radius, length):
            pass
    class cuboidal_beam:
        def __init__(self, geometry:np.ndarray):
            self.length = geometry[0]
            self.breadth = geometry[1]
            self.lbcons = geometry[2]
            self.constraint = geometry[3]
            if self.constraint == 'lb':
                self.lbcons()
        def lbcons():
            pass
   



     
    def __init__(self, type:str, geom:np.ndarray, load: np.ndarray, mat:np.ndarray):
        self.type = type
        if self.type == 'cubbeam':
            self.body = self.cuboidal_beam(geom)
        if load[0] =='ax':
            
        pass

class FE:
    def __init__(self, loc:np.ndarray):
        self.pos = loc
        self.state = [] 

In [31]:
class Material:
    def __init__(self, E:float, nu:float, rho:float):
        self.E = E 
        self.rho = rho
        self.nu = nu
        self.G = E / (2 * (1 + nu))

# class Beam:
#     def __init__(self, face_geom:np.ndarray):
#         if face_geom=='Rect':
#             return self.CuboidalBeam
class CuboidalBeam:
    def __init__(self, length:float, breadth:float, width:float):
        self.length = length
        self.breadth = breadth
        self.width = width
        self.facearea = breadth*width

    def constraint(self, constype: str = 'fixed', location: float = 'left'):
        """
        constype: 'fixed'
        location: 'left', 'right'
        """
        self.cons = [constype, location]

# class Load:
#     def __init__(self, loadtype: 'str', magnitude: float, location: float):
#         pass
#     pass

class AxialLoad:
    def __init__(self, value: float, location: float):
        '''Assume x along length, and give location in terms of x'''
        self.load = [value, location]
    pass

class Body:
    def __init__(self, material:Material, geometry:CuboidalBeam, load:AxialLoad, *args):
        self.material = material
        self.geometry = geometry
        self.load = load
        self.results = {}
        if isinstance(load, AxialLoad):
            self.axl()
    
    def axl(self,):
        # calc sigma_xx
        x = np.linspace(0, self.geometry.length, 100)
        if self.geometry.cons[1] == 'left':
            reaction = np.piecewise(
                x, 
                [(x>=0) & (x<=float(self.load.load[1]))], 
                [self.load.load[0]])

        if self.geometry.cons[1] == 'right':
            reaction = np.piecewise(
                x, 
                [(x >= float(self.load.load[1])) & (x <= float(self.geometry.length))], 
                [self.load.load[0]])
        
        axial_stress = reaction/(self.geometry.facearea)
        
        self.results['x'] = x
        self.results['axial_stress'] = axial_stress
        self.results['internal_force'] = reaction
    
    def plot_result(self,):
        if not self.results:
            raise ValueError("No results to plot. Run analysis first.")
        
        fig, axes = plt.subplots(2, 1, figsize=(10, 10))
        
        # Stress plot
        axes[0].plot(self.results['x'], self.results['axial_stress'] / 1e6, 'b-', linewidth=2)
        axes[0].set_xlabel('Position along beam (m)')
        axes[0].set_ylabel('Axial Stress (MPa)')
        axes[0].set_title('Axial Stress Distribution')
        axes[0].grid(True, alpha=0.3)
        # Internal force plot
        axes[1].plot(self.results['x'], self.results['internal_force'], 'r-', linewidth=2)
        axes[1].set_xlabel('Position along beam (m)')
        axes[1].set_ylabel('Internal Force (N)')
        axes[1].set_title('Internal Force Distribution')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    def print_results(self,):
        if not self.results:
            raise ValueError("No results to print. Run analysis first.")
        # print()
# class BoundaryConditions:
#     pass

In [2]:
# Define material (Steel)
steel = Material(E=200e9, nu=0.3, rho=7850)

# Define geometry
beam = CuboidalBeam(length=1.0, breadth=0.05, width=0.02)
beam.constraint(constype='fixed', location='left')

# Define load
load = AxialLoad(value=10000, location=0.5)  # 10 kN tension

# Create body and analyze
body = Body(material=steel, geometry=beam, load=load)

# Print summary
# body.print_summary()

# Plot results
body.plot_result()

NameError: name 'np' is not defined